<a href="https://colab.research.google.com/github/rajanchoksi79/pytorch_practice/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

print(torch.__version__)

2.6.0+cu124


In [1]:
print("hello")

hello
